In [1]:
import nltk
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import KFold, cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import metrics 
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import permutation_test_score
import re
import random
from sklearn.linear_model import LogisticRegression

/home/fatma/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/fatma/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [23]:
ready_cf_dataset_09_05_2018 = pd.read_csv("Data/ready_cf_dataset_09_05_2018.csv")

In [2]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /home/fatma/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " ", string)
    string = re.sub(r"\'ve", " ", string)
    string = re.sub(r"n\'t", " ", string)
    string = re.sub(r"n\'s", " ", string)
    string = re.sub(r"\'re", " ", string)
    string = re.sub(r"\'d", " ", string)
    string = re.sub(r"\'ll", " ", string)
    string = re.sub(r",", " ", string)
    string = re.sub(r"!", " ", string)
    string = re.sub(r"\(", " ", string)
    string = re.sub(r"\)", " ", string)
    string = re.sub(r"\?", " ", string)
    string = re.sub(r"\s{2,}", " ", string)
    string = re.sub(r"\//?", " ", string)
    string = re.sub(r"\d+", " ", string)
    string = re.sub(r"\$", " ", string)
    string = re.sub(r"\#", " ", string)
    return string.strip().lower()

In [4]:
def remove_stop(str):
    stop = set(stopwords.words('english'))
    lst = str.split()
    lst = [i for i in lst if i not in stop]
    return ' '.join(lst)

def stem (str):
    lst = str.split()
    lst = [st.stem(x) for x in lst]
    return ' '.join(lst)

def remove_unwanted_words(str):
    unwanted_words = ["httpaddress", "usrid", "D", "dd", "rt", "amp", "am", "pm", '``',
                      "''", "", "//", "\\", "\\'s", "\\?", "\?","http","httpaddresshttpaddresst", "cohttpaddressek",
                      "taksim","gezi", "park", "direngeziparki", "occupygezi", "istanbul", "turkish","turkey",
                      "protest","direngezipark","direnankara","geziparki", "protesters", "protests", "sat", "sun", "mon,",
                     "tue", "wed", "thu", "fri", "jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec"]
    lst = str.split(" ")
    lst = [i for i in lst if i not in unwanted_words]
    return ' '.join(lst)

def toLower(str):
    lst = str.split()
    lst = [i.lower() for i in lst]
    return ' '.join(lst)

def word_len (str):
    lst = str.split()
    lst = [i for i in lst if len(i)>1 and len(i) <7]
    return ' '.join(lst)

def sent_len (str):
    lst = str.split()
    if len(lst)>=3:
        return ' '.join(lst)

In [5]:
def load_data_and_labels_shuffled(positive_data_file, negative_data_file):
    """
    Loads MR polarity african_data from files, splits the african_data into words and generates labels.
    Returns split sentences and labels.
    """
    # Load african_data from files
    positive_examples = list(open(positive_data_file, "r").readlines())
    positive_examples = [s.strip() for s in positive_examples]
    positive_examples = [remove_stop(item) for item in positive_examples]
    #print(positive_examples[0])
    positive_examples = [toLower(item) for item in positive_examples]
    #rint(positive_examples[0])
    positive_examples = [clean_str(sent) for sent in positive_examples]
    #rint(positive_examples[0])
    #positive_examples = [stem(item) for item in positive_examples]
    #rint(positive_examples[0])
    positive_examples = [remove_unwanted_words(item) for item in positive_examples]
    positive_examples = [word_len(item) for item in positive_examples]
    positive_examples = [sent_len(item) for item in positive_examples]
    positive_examples = list(filter(None, positive_examples))
    negative_examples = list(open(negative_data_file, "r").readlines())
    negative_examples = [s.strip() for s in negative_examples]
    negative_examples = [toLower(item) for item in negative_examples]
    #print(negative_examples[0])
    negative_examples = [remove_stop(item) for item in negative_examples]
    #print(negative_examples[0])
    negative_examples = [clean_str(sent) for sent in negative_examples]
    #print(negative_examples[0])
    #ngative_examples = [stem(sent) for sent in negative_examples]
    #pint(negative_examples[0])
    negative_examples = [remove_unwanted_words(item) for item in negative_examples]
    negative_examples = [word_len(item) for item in negative_examples]
    #print(negative_examples[0])
    negative_examples = [sent_len(item) for item in negative_examples]

    negative_examples = list(filter(None, negative_examples))

    # Split by words
    x_text = positive_examples + negative_examples
    # x_text = [clean_str(sent) for sent in x_text]
    # Generate labels
    positive_labels = [1 for _ in positive_examples]
    negative_labels = [0 for _ in negative_examples]
    y = np.concatenate([positive_labels, negative_labels], 0)
    return [x_text, y]

In [44]:
len(ready_cf_dataset_09_05_2018)

1735

In [36]:
len(ready_cf_dataset_09_05_2018[ready_cf_dataset_09_05_2018["violence"]==1])

205

In [37]:
len(ready_cf_dataset_09_05_2018[ready_cf_dataset_09_05_2018["violence"]==0])

1530

In [24]:
X_train, X_test, y_train, y_test = train_test_split(ready_cf_dataset_09_05_2018.proccd_text, 
                                                    ready_cf_dataset_09_05_2018.violence, 
                                                    test_size=0.3, random_state=0)

In [25]:
train_df = pd.concat([X_train,y_train],axis=1)
test_df = pd.concat([X_test,y_test],axis = 1)

In [45]:
len(train_df)

1214

In [38]:
len(train_df[train_df["violence"]==1])

151

In [40]:
len(train_df[train_df["violence"]==0])

1063

In [46]:
len(test_df)

521

In [42]:
len(test_df[test_df["violence"]==1])

54

In [43]:
len(test_df[test_df["violence"]==0])

467

In [26]:
pipeline_violence_Svc = Pipeline([
    ('vectorizer',  CountVectorizer(ngram_range=(1, 2),stop_words='english',  min_df=3)),
    ('tfidf_transformer',  TfidfTransformer()),
    ('classifier',  SVC(kernel='linear', gamma=2)) ])

    
scores = cross_val_score(pipeline_violence_Svc, train_df.proccd_text, train_df.violence, cv=5)

In [56]:
print('scores',scores)
print('scores mean',scores.mean())
print('scores std',scores.std() * 2)

('scores', array([ 0.86885246,  0.86831276,  0.89300412,  0.88429752,  0.88842975]))
('scores mean', 0.88057932083432999)
('scores std', 0.020353233995855032)


In [51]:
cv_predict = cross_val_predict(pipeline_violence_Svc, train_df.proccd_text, train_df.violence, cv=5)
cv_predict_scores = metrics.roc_auc_score(train_df.violence, cv_predict)

In [52]:
cv_predict_scores

0.59941250864415974

In [28]:
pipeline_violence_Svc.fit(train_df.proccd_text, train_df.violence)

Pipeline(steps=[('vectorizer', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=3,
        ngram_range=(1, 2), preprocessor=None, stop_words='english',...,
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])

In [29]:
prediction = pipeline_violence_Svc.predict(test_df.proccd_text)

In [32]:
metrics.roc_auc_score(test_df.violence,prediction)

0.58402728210008714

In [33]:
metrics.confusion_matrix(test_df.violence,prediction)

array([[459,   8],
       [ 44,  10]])

# svm on the same DS used in dann_tf_code

In [8]:
text, y = load_data_and_labels_shuffled('Data/Turkish_tweets_CF_results_09_05_2018_prccd_pos.txt', 'Data/Turkish_tweets_CF_results_09_05_2018_prccd_neg.txt')

In [9]:
df = pd.DataFrame({'proccd_text': list(text),'violence': y})

In [10]:
len(df)

1333

In [11]:
len(df[df["violence"]==1])

180

In [12]:
len(df[df["violence"]==0])

1153

In [13]:
df_X_train, df_X_test, df_y_train, df_y_test = train_test_split(df.proccd_text, 
                                                    df.violence, 
                                                    test_size=0.3, random_state=0)

In [14]:
df_train_df = pd.concat([df_X_train,df_y_train],axis=1)
df_test_df = pd.concat([df_X_test,df_y_test],axis = 1)

In [15]:
len(df_train_df[df_train_df["violence"]==1])


135

In [16]:
len(df_train_df[df_train_df["violence"]==0])

798

In [17]:
len(df_test_df)

400

In [18]:

len(df_test_df[df_test_df["violence"]==1])


45

In [19]:

len(df_test_df[df_test_df["violence"]==0])

355

In [20]:
df_pipeline_violence_Svc = Pipeline([
    ('vectorizer',  CountVectorizer(ngram_range=(1, 2),stop_words='english',  min_df=3)),
    ('classifier',  SVC(kernel='linear', gamma=2)) ])

    
df_scores = cross_val_score(df_pipeline_violence_Svc, df_train_df.proccd_text, df_train_df.violence, cv=5)

In [21]:
print('scores',df_scores)
print('scores mean',df_scores.mean())
print('scores std',df_scores.std() * 2)

('scores', array([ 0.90374332,  0.86631016,  0.86096257,  0.8172043 ,  0.89247312]))
('scores mean', 0.8681386924271175)
('scores std', 0.060060009965560136)


In [22]:
df_cv_predict = cross_val_predict(df_pipeline_violence_Svc, df_train_df.proccd_text, df_train_df.violence, cv=5)
df_cv_predict_scores = metrics.roc_auc_score(df_train_df.violence, df_cv_predict)

In [23]:
df_cv_predict_scores

0.66752993595098864

In [24]:
df_pipeline_violence_Svc.fit(df_train_df.proccd_text, df_train_df.violence)

Pipeline(steps=[('vectorizer', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=3,
        ngram_range=(1, 2), preprocessor=None, stop_words='english',...,
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])

In [25]:
df_prediction = df_pipeline_violence_Svc.predict(df_test_df.proccd_text)

In [26]:
metrics.roc_auc_score(df_test_df.violence,df_prediction)

0.6912363067292645

In [27]:
metrics.confusion_matrix(df_test_df.violence,df_prediction)

array([[333,  22],
       [ 25,  20]])

# Logistic regreswsion

In [28]:
lr_pipeline_violence_Svc = Pipeline([
    ('vectorizer',  CountVectorizer(ngram_range=(1, 2),stop_words='english',  min_df=3)),
    ('classifier',  LogisticRegression()) ])

    
lr_scores = cross_val_score(lr_pipeline_violence_Svc, df_train_df.proccd_text, df_train_df.violence, cv=5)

In [29]:
print('scores',lr_scores)
print('scores mean',lr_scores.mean())
print('scores std',lr_scores.std() * 2)

('scores', array([ 0.90374332,  0.87700535,  0.87700535,  0.86021505,  0.88172043]))
('scores mean', 0.87993789891323093)
('scores std', 0.027943737931744168)


In [30]:
lr_cv_predict = cross_val_predict(lr_pipeline_violence_Svc, df_train_df.proccd_text, df_train_df.violence, cv=5)
lr_cv_predict_scores = metrics.roc_auc_score(df_train_df.violence, lr_cv_predict)

In [31]:
lr_cv_predict_scores

0.65595934280144808

In [32]:
lr_pipeline_violence_Svc.fit(df_train_df.proccd_text, df_train_df.violence)

Pipeline(steps=[('vectorizer', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=3,
        ngram_range=(1, 2), preprocessor=None, stop_words='english',...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [33]:
lr_prediction = lr_pipeline_violence_Svc.predict(df_test_df.proccd_text)

In [34]:
metrics.roc_auc_score(df_test_df.violence,lr_prediction)

0.65805946791862291

In [35]:
metrics.confusion_matrix(df_test_df.violence,lr_prediction)

array([[341,  14],
       [ 29,  16]])